# Capstone Defense Scheduling Algorithm

This notebook implements a comprehensive algorithm to match **capstone defenses with panelists** and schedule them into **1-hour calendar slots**. The algorithm:

1. **Groups panelists by topics** - Ensures topic compatibility between projects and panelists
2. **Assigns panelists to projects** - Matches panelists based on expertise while respecting constraints
3. **Schedules into calendar slots** - Assigns defenses to available time slots

## Constraints:
- ✅ Topic compatibility (panelists must have expertise in project topic)
- ✅ Supervisor exclusion (supervisors cannot be on their own project's panel)
- ✅ Panelist capacity limits (respects maximum number of panels per panelist)
- ✅ Panelist availability (no double booking in time slots)
- ✅ Each project gets exactly the required number of panelists
- ✅ Each project is scheduled exactly once


In [26]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
from collections import defaultdict
from typing import Dict, List, Tuple, Optional

# Import the scheduler module
from capstone_scheduler import (
    group_panelists_by_topics,
    assign_panelists_to_projects,
    schedule_defenses,
    match_defenses_and_panelists,
    print_summary_report
)


## Input Data

Define your projects, panelists, topics, time slots, and availability.


In [33]:
# LARGER WORKING EXAMPLE
# 8 projects, 10 panelists, 4 topics, 12 time slots
# This example demonstrates a more realistic capstone defense scheduling scenario

projects = pd.DataFrame({
    "project_id": ["P01", "P02", "P03", "P04", "P05", "P06", "P07", "P08"],
    "topic": ["NLP", "Finance", "ML", "NLP", "Finance", "ML", "NLP", "Finance"],
    "supervisor": ["Prof_A", "Prof_B", "Prof_C", "Prof_D", "Prof_E", "Prof_F", "Prof_G", "Prof_H"],
    "required_panelists": [2, 2, 2, 2, 2, 2, 2, 2]
})

# 10 panelists with varying capacity
panelists = pd.DataFrame({
    "panelist_id": ["Prof_A", "Prof_B", "Prof_C", "Prof_D", "Prof_E", 
                    "Prof_F", "Prof_G", "Prof_H", "Prof_I", "Prof_J"],
    "max_panels": [4, 4, 3, 3, 3, 3, 2, 2, 2, 2]
})

# Panelist expertise matrix: 1 = has expertise, 0 = no expertise
# Multiple panelists per topic to ensure feasibility
panelist_topics = pd.DataFrame({
    "panelist_id": ["Prof_A", "Prof_B", "Prof_C", "Prof_D", "Prof_E", 
                    "Prof_F", "Prof_G", "Prof_H", "Prof_I", "Prof_J"],
    "NLP": [1, 0, 0, 1, 0, 0, 1, 0, 1, 1],      # 5 panelists: A, D, G, I, J
    "Finance": [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],  # 5 panelists: B, E, H, I
    "ML": [0, 0, 1, 1, 0, 1, 1, 0, 0, 1],       # 5 panelists: C, D, F, G, J
    "Data_Science": [1, 1, 1, 0, 0, 0, 0, 1, 1, 0]  # 5 panelists: A, B, C, H, I
})

# 12 time slots across 3 days (4 slots per day)
slots = pd.DataFrame({
    "slot_id": ["S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10", "S11", "S12"],
    "date": ["2026-06-12", "2026-06-12", "2026-06-12", "2026-06-12",
             "2026-06-13", "2026-06-13", "2026-06-13", "2026-06-13",
             "2026-06-14", "2026-06-14", "2026-06-14", "2026-06-14"],
    "time": ["09-10", "10-11", "11-12", "14-15",
             "09-10", "10-11", "11-12", "14-15",
             "09-10", "10-11", "11-12", "14-15"],
    "room": ["R1", "R1", "R2", "R2", "R1", "R1", "R2", "R2", "R1", "R1", "R2", "R2"]
})

# Panelist availability: Realistic pattern with some conflicts
# 1 = available, 0 = not available
availability = pd.DataFrame({
    "panelist_id": ["Prof_A", "Prof_B", "Prof_C", "Prof_D", "Prof_E", 
                    "Prof_F", "Prof_G", "Prof_H", "Prof_I", "Prof_J"],
    "S01": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 1 morning - all available
    "S02": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 1 late morning
    "S03": [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],  # Day 1 early afternoon (Prof_C unavailable)
    "S04": [1, 1, 1, 1, 1, 0, 1, 1, 1, 1],  # Day 1 afternoon (Prof_F unavailable)
    "S05": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 2 morning - all available
    "S06": [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 2 late morning (Prof_B unavailable)
    "S07": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 2 early afternoon
    "S08": [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],  # Day 2 afternoon (Prof_D unavailable)
    "S09": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 3 morning - all available
    "S10": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 3 late morning
    "S11": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # Day 3 early afternoon
    "S12": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   # Day 3 afternoon
})

print("="*80)
print("LARGER EXAMPLE DATA")
print("="*80)
print(f"\n📊 Summary:")
print(f"  • {len(projects)} projects across {projects.topic.nunique()} topics")
print(f"  • {len(panelists)} panelists")
print(f"  • {len(slots)} time slots across {slots.date.nunique()} days")
print(f"  • Total panelist slots needed: {projects.required_panelists.sum()}")
print(f"  • Total panelist capacity: {panelists.max_panels.sum()}")

print("\n" + "="*80)
print("PROJECTS")
print("="*80)
print(projects.to_string(index=False))

print("\n" + "="*80)
print("PANELISTS")
print("="*80)
print(panelists.to_string(index=False))

print("\n" + "="*80)
print("PANELIST EXPERTISE")
print("="*80)
print(panelist_topics.to_string(index=False))

print("\n" + "="*80)
print("TIME SLOTS")
print("="*80)
print(slots.to_string(index=False))

print("\n" + "="*80)
print("FEASIBILITY CHECK")
print("="*80)
all_feasible = True
for _, p in projects.iterrows():
    topic = p.topic
    supervisor = p.supervisor
    required = p.required_panelists
    eligible = []
    for j in panelists.panelist_id:
        if j == supervisor:
            continue
        expertise = panelist_topics.loc[panelist_topics.panelist_id == j, topic].values
        if len(expertise) > 0 and int(expertise[0]) == 1:
            eligible.append(j)
    status = "✅" if len(eligible) >= required else "❌"
    if len(eligible) < required:
        all_feasible = False
    print(f"{status} {p.project_id} ({topic}): needs {required}, has {len(eligible)} eligible: {eligible}")

if all_feasible:
    print("\n✅ All projects have sufficient eligible panelists!")
else:
    print("\n❌ Some projects need more eligible panelists")
print("="*80)


LARGER EXAMPLE DATA

📊 Summary:
  • 8 projects across 3 topics
  • 10 panelists
  • 12 time slots across 3 days
  • Total panelist slots needed: 16
  • Total panelist capacity: 28

PROJECTS
project_id   topic supervisor  required_panelists
       P01     NLP     Prof_A                   2
       P02 Finance     Prof_B                   2
       P03      ML     Prof_C                   2
       P04     NLP     Prof_D                   2
       P05 Finance     Prof_E                   2
       P06      ML     Prof_F                   2
       P07     NLP     Prof_G                   2
       P08 Finance     Prof_H                   2

PANELISTS
panelist_id  max_panels
     Prof_A           4
     Prof_B           4
     Prof_C           3
     Prof_D           3
     Prof_E           3
     Prof_F           3
     Prof_G           2
     Prof_H           2
     Prof_I           2
     Prof_J           2

PANELIST EXPERTISE
panelist_id  NLP  Finance  ML  Data_Science
     Prof_A    1     

## Step 1: Group Panelists by Topics

First, let's analyze and group panelists by their topic expertise.


In [28]:
# Group panelists by topics (using imported function)
topic_groups = group_panelists_by_topics(panelist_topics)
print("Panelists grouped by topics:")
for topic, panelist_list in topic_groups.items():
    print(f"  {topic}: {', '.join(panelist_list)}")


Panelists grouped by topics:
  NLP: Prof_A, Prof_D, Prof_G, Prof_I, Prof_J
  Data_Science: Prof_A, Prof_B, Prof_C, Prof_H, Prof_I
  Finance: Prof_B, Prof_E, Prof_H, Prof_I
  ML: Prof_C, Prof_D, Prof_F, Prof_G, Prof_J


## Step 2: Panel Assignment (MILP)

Assign panelists to projects using Mixed Integer Linear Programming, ensuring:
- Topic compatibility
- Supervisor exclusion  
- Panelist capacity limits


# Run panel assignment
panel_assignment, success = assign_panelists_to_projects(projects, panelists, panelist_topics)

if success:
    print("Panel Assignment Results:")
    print(panel_assignment)
    
    # Show assignments grouped by project
    print("\nAssignments by Project:")
    for project_id in projects.project_id:
        assigned = panel_assignment[panel_assignment.project_id == project_id]
        if len(assigned) > 0:
            panelists_list = assigned.panelist_id.tolist()
            project_topic = projects.loc[projects.project_id == project_id, "topic"].values[0]
            print(f"  {project_id} ({project_topic}): {', '.join(panelists_list)}")
else:
    print("Failed to assign panelists. Please check constraints.")


In [29]:
panel_assignment, success = assign_panelists_to_projects(projects, panelists, panelist_topics)

if success:
    print("Panel Assignment Results:")
    print(panel_assignment)
    
    # Show assignments grouped by project
    print("\nAssignments by Project:")
    for project_id in projects.project_id:
        assigned = panel_assignment[panel_assignment.project_id == project_id]
        if len(assigned) > 0:
            panelists_list = assigned.panelist_id.tolist()
            project_topic = projects.loc[projects.project_id == project_id, "topic"].values[0]
            print(f"  {project_id} ({project_topic}): {', '.join(panelists_list)}")
else:
    print("Failed to assign panelists. Please check constraints.")

Panel Assignment Results:
   project_id panelist_id
0         P01      Prof_I
1         P01      Prof_J
2         P02      Prof_E
3         P02      Prof_H
4         P03      Prof_D
5         P03      Prof_F
6         P04      Prof_A
7         P04      Prof_G
8         P05      Prof_B
9         P05      Prof_H
10        P06      Prof_D
11        P06      Prof_G
12        P07      Prof_A
13        P07      Prof_D
14        P08      Prof_B
15        P08      Prof_E

Assignments by Project:
  P01 (NLP): Prof_I, Prof_J
  P02 (Finance): Prof_E, Prof_H
  P03 (ML): Prof_D, Prof_F
  P04 (NLP): Prof_A, Prof_G
  P05 (Finance): Prof_B, Prof_H
  P06 (ML): Prof_D, Prof_G
  P07 (NLP): Prof_A, Prof_D
  P08 (Finance): Prof_B, Prof_E


## Step 3: Schedule Defenses into Calendar Slots

Assign projects to time slots, ensuring:
- Each project is scheduled exactly once
- All assigned panelists are available
- No double booking (panelists can't be in two places at once)

In [30]:
# Run scheduling
if success:
    schedule, schedule_success = schedule_defenses(
        projects, panel_assignment, slots, availability
    )
    
    if schedule_success:
        print("Schedule Results:")
        print(schedule.to_string(index=False))
    else:
        print("Failed to create schedule. Please check availability constraints.")
else:
    print("Cannot schedule: panel assignment failed.")


Schedule Results:
slot_id       date  time room project_id   topic      panelists  num_panelists
    S01 2026-06-12 09-10   R1        P01     NLP Prof_I, Prof_J              2
    S06 2026-06-13 10-11   R1        P02 Finance Prof_E, Prof_H              2
    S03 2026-06-12 11-12   R2        P03      ML Prof_D, Prof_F              2
    S08 2026-06-13 14-15   R2        P04     NLP Prof_A, Prof_G              2
    S01 2026-06-12 09-10   R1        P05 Finance Prof_B, Prof_H              2
    S01 2026-06-12 09-10   R1        P06      ML Prof_D, Prof_G              2
    S02 2026-06-12 10-11   R1        P07     NLP Prof_A, Prof_D              2
    S02 2026-06-12 10-11   R1        P08 Finance Prof_B, Prof_E              2


## Step 4: Summary Reports

Generate summary reports grouped by topics and calendar view.


In [31]:
# Summary: Panelists grouped by topics with their assignments
# Check if all required variables exist
if 'success' in locals() and 'schedule_success' in locals() and success and schedule_success:
    if 'topic_groups' in locals() and 'panel_assignment' in locals() and 'schedule' in locals():
        print_summary_report({
            "topic_groups": topic_groups,
            "panel_assignment": panel_assignment,
            "schedule": schedule,
            "success": True
        }, projects)
    else:
        print("Missing required variables. Please run previous cells first.")
elif 'success' in locals() and not success:
    print("Cannot generate summary: panel assignment failed.")
elif 'schedule_success' in locals() and not schedule_success:
    print("Cannot generate summary: scheduling failed.")
else:
    print("Cannot generate summary: please run cells 7 and 9 first.")


SUMMARY: PANELISTS GROUPED BY TOPICS

📚 Topic: NLP
--------------------------------------------------------------------------------
Available Panelists: Prof_A, Prof_D, Prof_G, Prof_I, Prof_J
Projects: P01, P04, P07

Assignments:
  • P01 → Prof_I (Scheduled: 2026-06-12 09-10 in R1)
  • P01 → Prof_J (Scheduled: 2026-06-12 09-10 in R1)
  • P04 → Prof_A (Scheduled: 2026-06-13 14-15 in R2)
  • P04 → Prof_G (Scheduled: 2026-06-13 14-15 in R2)
  • P07 → Prof_A (Scheduled: 2026-06-12 10-11 in R1)
  • P07 → Prof_D (Scheduled: 2026-06-12 10-11 in R1)

Scheduled Defenses:
  • 2026-06-12 09-10 | Room: R1 | Project: P01
    Panelists: Prof_I, Prof_J
  • 2026-06-13 14-15 | Room: R2 | Project: P04
    Panelists: Prof_A, Prof_G
  • 2026-06-12 10-11 | Room: R1 | Project: P07
    Panelists: Prof_A, Prof_D

📚 Topic: Finance
--------------------------------------------------------------------------------
Available Panelists: Prof_B, Prof_E, Prof_H, Prof_I
Projects: P02, P05, P08

Assignments:
  • P02 → P

## Complete Algorithm Function

Run the complete algorithm in one step:


In [32]:
# Run the complete algorithm
result = match_defenses_and_panelists(
    projects, panelists, panelist_topics, slots, availability
)

if result["success"]:
    print("✅ Algorithm completed successfully!")
    print(f"\nTopic Groups: {len(result['topic_groups'])} topics")
    print(f"Panel Assignments: {len(result['panel_assignment'])} assignments")
    print(f"Scheduled Defenses: {len(result['schedule'])} defenses")
    
    # Display results
    print("\n" + "="*80)
    print("PANEL ASSIGNMENTS")
    print("="*80)
    print(result['panel_assignment'])
    
    print("\n" + "="*80)
    print("SCHEDULE")
    print("="*80)
    print(result['schedule'])
    
    print("\n" + "="*80)
    print("SUMMARY BY TOPICS")
    print("="*80)
    print_summary_report(result, projects)
else:
    print("❌ Algorithm failed. Check constraints and data.")


✅ Algorithm completed successfully!

Topic Groups: 4 topics
Panel Assignments: 16 assignments
Scheduled Defenses: 8 defenses

PANEL ASSIGNMENTS
   project_id panelist_id
0         P01      Prof_I
1         P01      Prof_J
2         P02      Prof_E
3         P02      Prof_H
4         P03      Prof_D
5         P03      Prof_F
6         P04      Prof_A
7         P04      Prof_G
8         P05      Prof_B
9         P05      Prof_H
10        P06      Prof_D
11        P06      Prof_G
12        P07      Prof_A
13        P07      Prof_D
14        P08      Prof_B
15        P08      Prof_E

SCHEDULE
  slot_id        date   time room project_id    topic       panelists  \
0     S01  2026-06-12  09-10   R1        P01      NLP  Prof_I, Prof_J   
1     S06  2026-06-13  10-11   R1        P02  Finance  Prof_E, Prof_H   
2     S03  2026-06-12  11-12   R2        P03       ML  Prof_D, Prof_F   
3     S08  2026-06-13  14-15   R2        P04      NLP  Prof_A, Prof_G   
4     S01  2026-06-12  09-10   R1      